## Importing libraries

In [20]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.layers import (
    Dense,
    Flatten,
    Dropout,
    Conv2D,
    MaxPooling2D,
    Activation,
    BatchNormalization
)
from keras.utils import np_utils
from keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

## Reading and loading data

In [21]:
sample_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
train = pd.read_csv("../input/digit-recognizer/train.csv")

In [22]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Preparing the test and train data

In [23]:
X_train = train.loc[:, train.columns!='label'].values.astype('uint8')
y_train = train['label'].values

X_train = X_train.reshape((X_train.shape[0],28,28))

In [24]:
X_test = test.loc[:, test.columns!='label'].values.astype('uint8')
X_test = X_test.reshape((X_test.shape[0],28,28))

Reshaping the data, So that it fits in keras image format : (samples, rows, cols, channels)

In [25]:
X_train = X_train[:,:,:,None]
X_test = X_test[:,:,:,None]

In [26]:
X_train.shape

(42000, 28, 28, 1)

## Defining some parameters:

In [27]:
batch_size = 32
num_samples = X_train.shape[0]
num_classes = np.unique(y_train).shape[0]
num_epochs = 50
img_rows, img_cols = X_train[0,:,:,0].shape
img_channels = 1
classes = np.unique(y_train)

In [28]:
y_train = np_utils.to_categorical(y_train, num_classes)
# y_test = np_utils.to_categorical(y_test, num_classes)

## Standardizing the data:

In [29]:
X_train_norm = X_train.astype('float32')
X_test_norm = X_test.astype('float32')
X_train_norm /= 255
X_test_norm /= 255

## Model

In [30]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [32]:
history = model.fit(
    X_train_norm,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_split=0.1,
    shuffle=True,
    callbacks=[es]
)

Train on 37800 samples, validate on 4200 samples
Epoch 1/50
37800/37800 [==============================] - 9s 250us/step - loss: 0.0438 - accuracy: 0.9849 - val_loss: 0.0132 - val_accuracy: 0.9951
Epoch 2/50
37800/37800 [==============================] - 6s 171us/step - loss: 0.0177 - accuracy: 0.9942 - val_loss: 0.0079 - val_accuracy: 0.9968
Epoch 3/50
37800/37800 [==============================] - 6s 165us/step - loss: 0.0131 - accuracy: 0.9959 - val_loss: 0.0077 - val_accuracy: 0.9972
Epoch 4/50
37800/37800 [==============================] - 6s 171us/step - loss: 0.0111 - accuracy: 0.9964 - val_loss: 0.0067 - val_accuracy: 0.9974
Epoch 5/50
37800/37800 [==============================] - 7s 176us/step - loss: 0.0093 - accuracy: 0.9969 - val_loss: 0.0062 - val_accuracy: 0.9981
Epoch 6/50
37800/37800 [==============================] - 6s 171us/step - loss: 0.0083 - accuracy: 0.9973 - val_loss: 0.0069 - val_accuracy: 0.9976
Epoch 7/50
37800/37800 [==============================] - 7s 17

In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [ ]:
model.save('newer/simple.h5')

### Looks like the last variant was the best one

In [34]:
pred = model.predict_classes(X_test_norm)

In [35]:
sample_submission['Label'] = pred

In [36]:
sample_submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [37]:
sample_submission.to_csv("submission.csv", index=False)